IMPORTS

In [1]:
import numpy as np
import pandas as pd
import shapely
import geopandas as gpd
import pyarrow.parquet as pq

from shapely.geometry import mapping, shape
from rasterio.mask import mask
import rasterio
from tqdm import tqdm

import geojson
import keras
import joblib

c:\Users\A44505693\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
name = "Madhya_Pradesh"
tif_tile = r"GHS_SMOD_E2030_GLOBE_R2023A_MP_Merged.tif"
state_polygon = r"Madhya_Pradesh.geojson"
parquet_path = r"FEATURES_DB_MADHYA_PRADESH.parquet"
model_file_name = r"BN_informal_settlements_models\informal_settlements_model_wotrees_50m_all_tiles.keras"

SMOD CUT

In [3]:
state_gdf = gpd.read_file(state_polygon)


with rasterio.open(tif_tile) as src:
    raster_crs = src.crs
    if state_gdf.crs != raster_crs:
        state_gdf = state_gdf.to_crs(raster_crs)

    state_geom = [state_gdf.geometry.iloc[0]]
    out_image, out_transform = rasterio.mask.mask(src, state_geom, crop=True)

#urban_mask = ((out_image == 30) | (out_image == 23)).astype("uint8") 
urban_mask = (out_image == 30).astype("uint8")
features = []
for s, v in rasterio.features.shapes(urban_mask, transform=out_transform):
    if v == 1:
        features.append({"geometry": shape(s), "value": v})

urban_gdf = gpd.GeoDataFrame(features, geometry="geometry", crs=state_gdf.crs)


urban_gdf = urban_gdf.to_crs(epsg=4326)
urban_gdf.to_file(f"urban_centers_{name}.geojson", driver="GeoJSON")



CUTTING TO SQUARES

In [4]:
class GridGenerator:

    def rectangles_inside_polygon(self, polygon, n=None, size=None, tol=0, clip=True, include_poly=False) -> gpd.geoseries.GeoSeries:
            assert (n is None and size is not None) or (n is not None and size is None)
            # Extract bounding box coordinates of the polygon
            
            a, b, c, d = gpd.GeoSeries(polygon).total_bounds
            

            # Generate grids along x-axis/y-axis on the n or size
            if not n is None:
                xa = np.linspace(a, c, n + 1)
                ya = np.linspace(b, d, n + 1)
            else:
                xa = np.arange(a, c + 1, size[0])
                ya = np.arange(b, d + 1, size[1])

            # Offsets for tolerance to prevent edge cases
            if tol != 0:
                tol_xa = np.arange(0, tol * len(xa), tol)
                tol_ya = np.arange(0, tol * len(ya), tol)

            else:
                tol_xa = np.zeros(len(xa))
                tol_ya = np.zeros(len(ya))

            # Combine placements of x&y with tolerance
            xat = np.repeat(xa, 2)[1:] + np.repeat(tol_xa, 2)[:-1]
            yat = np.repeat(ya, 2)[1:] + np.repeat(tol_ya, 2)[:-1]

            # Create a grid
            grid = gpd.GeoSeries(
                [
                    shapely.geometry.box(minx, miny, maxx, maxy)
                    for minx, maxx in xat[:-1].reshape(len(xa) - 1, 2)
                    for miny, maxy in yat[:-1].reshape(len(ya) - 1, 2)
                ]
            )

            # Ensure all returned polygons are within boundary
            if clip:
                # grid = grid.loc[grid.within(gpd.GeoSeries(np.repeat([polygon], len(grid))))]
                grid = gpd.sjoin(
                    gpd.GeoDataFrame(geometry=grid),
                    gpd.GeoDataFrame(geometry=[polygon]),
                    how="inner",
                    predicate="within",
                )["geometry"]
            # useful for visualisation
            if include_poly:
                grid = pd.concat(
                    [
                        grid,
                        gpd.GeoSeries(
                            polygon.geoms
                            if isinstance(polygon, shapely.geometry.MultiPolygon)
                            else polygon
                        ),
                    ]
                )
            return grid

    def rectangles_inside_geojson(geojson_data, size):
        """
        Generate a grid of rectangles fully inside all rectangular polygons in a GeoJSON FeatureCollection.

        Args:
            geojson_data (dict): GeoJSON FeatureCollection with rectangular polygons.
            size (tuple): (width, height) of each rectangle.

        Returns:
            gpd.GeoSeries: Grid of rectangles clipped to the original polygons.
        """
        dx, dy = size
        all_polygons = []

        # Extract all polygons/multipolygons from GeoJSON
        for feature in geojson_data.get("features", []):
            geom_type = feature["geometry"]["type"]
            coords = feature["geometry"]["coordinates"]

            if geom_type == "Polygon":
                all_polygons.append(shapely.geometry.Polygon(coords[0]))
            elif geom_type == "MultiPolygon":
                all_polygons.extend([shapely.geometry.Polygon(p[0]) for p in coords])

        # Combine into a single GeoDataFrame
        poly_gdf = gpd.GeoDataFrame(geometry=all_polygons)

        # Generate rectangles for each polygon's bounding box
        rectangles = []
        for poly in all_polygons:
            minx, miny, maxx, maxy = poly.bounds
            x_coords = np.arange(minx, maxx, dx)
            y_coords = np.arange(miny, maxy, dy)

            for x in x_coords:
                for y in y_coords:
                    rect = shapely.geometry.box(x, y, min(x + dx, maxx), min(y + dy, maxy))
                    rectangles.append(rect)

        # Build GeoDataFrame of all rectangles
        rect_gdf = gpd.GeoDataFrame(geometry=rectangles)

        # Clip using spatial join (returns only those within any polygon)
        clipped = gpd.sjoin(rect_gdf, poly_gdf, how="inner", predicate="within")

        return clipped["geometry"]

In [5]:
def prepare_data(file):
    # Read JSON
    gdf = gpd.read_file(file)
    # print(gdf["geometry"])

    records = []

    for i, polygon in enumerate(gdf["geometry"]):
        # Get bounds: [minx, miny, maxx, maxy]
        minx, miny, maxx, maxy = polygon.bounds
        records.append(
            {
                "id": str(minx) + ":" + str(miny),
                "geometry": polygon,
                "min_lon": minx,
                "max_lon": maxx,
                "min_lat": miny,
                "max_lat": maxy,
            }
        )

    # Create new GeoDataFrame with geometry and bounds
    bounds_gdf = gpd.GeoDataFrame(records)

    return bounds_gdf

In [6]:
def convert_polygon(polygon):
    # Convert WKT to Shapely geometry
    from shapely.geometry import mapping

    # Convert to GeoJSON dict
    geojson_obj = geojson.Feature(geometry=mapping(polygon))

    # Print or export to paste into geojson.io
    #print(geojson.dumps(geojson_obj))

    return geojson.dumps(geojson_obj)

In [7]:
grid = GridGenerator()
#!!!!!!!define polygon(s) to prepare in geojson format
file = f"urban_centers_{name}.geojson"
#!!!!!!!define type of data formal/informal/real - infromal from shelter associates are cut and prepared 
classification = "real"
#50m squares all informal settlements.json
#100m squares all informal settlements.json
#!!!!!!!define cut size 50mx50m or 100mx100m in int format 50/100 (in case of different size, values in next steps (lat,lon) needs to be changed/added)
cut_size = 50

In [8]:
#read the choosen area to cut into squares in geojson format
polygons = gpd.read_file(file)
all_grids = []
polygons

,value,geometry
0,1.0,"POLYGON ((78.23942 26.72336, 78.23581 26.71492..."
1,1.0,"POLYGON ((78.34990 26.70647, 78.33910 26.68115..."
2,1.0,"POLYGON ((78.77365 26.59673, 78.74845 26.53766..."
3,1.0,"POLYGON ((78.59873 26.51235, 78.59515 26.50391..."
4,1.0,"POLYGON ((78.01445 26.52078, 78.01090 26.51235..."
...,...,...
95,1.0,"POLYGON ((75.09996 21.71322, 75.09458 21.69658..."
96,1.0,"POLYGON ((78.80327 21.67163, 78.80045 21.66331..."
97,1.0,"POLYGON ((78.50942 21.60510, 78.50662 21.59678..."
98,1.0,"POLYGON ((80.39762 21.38895, 80.39479 21.38064..."


In [9]:
#cutting into squares - size defined in 1st tile
total = len(polygons)

for idx, polygon in enumerate(polygons["geometry"]):
    progress = f"{idx + 1}/{total}"
    print(f"Processing polygon {progress}...")

    if cut_size == 50:
        inside_grid = grid.rectangles_inside_polygon(polygon=polygon, size=(0.000451369, 0.00045121))
    elif cut_size == 100:
        inside_grid = grid.rectangles_inside_polygon(polygon=polygon, size=(0.000902738, 0.00090242))

    all_grids.extend(list(inside_grid))

gdf = gpd.GeoDataFrame(all_grids, columns=["geometry"])
cut_file_name = f'{file}_cut_{cut_size}m.geojson'
cut_file = gdf.to_file(cut_file_name, driver='GeoJSON')

Processing polygon 1/100...
Processing polygon 2/100...
Processing polygon 3/100...
Processing polygon 4/100...
Processing polygon 5/100...
Processing polygon 6/100...
Processing polygon 7/100...
Processing polygon 8/100...
Processing polygon 9/100...
Processing polygon 10/100...
Processing polygon 11/100...
Processing polygon 12/100...
Processing polygon 13/100...
Processing polygon 14/100...
Processing polygon 15/100...
Processing polygon 16/100...
Processing polygon 17/100...
Processing polygon 18/100...
Processing polygon 19/100...
Processing polygon 20/100...
Processing polygon 21/100...
Processing polygon 22/100...
Processing polygon 23/100...
Processing polygon 24/100...
Processing polygon 25/100...
Processing polygon 26/100...
Processing polygon 27/100...
Processing polygon 28/100...
Processing polygon 29/100...
Processing polygon 30/100...
Processing polygon 31/100...
Processing polygon 32/100...
Processing polygon 33/100...
Processing polygon 34/100...
Processing polygon 35/1

In [10]:
cut_file_name = f'{file}_cut_{cut_size}m.geojson'
grid_gdf = gpd.read_file(cut_file_name)

In [11]:
grid_gdf.insert(0, "minx", 0)
grid_gdf.insert(1, "miny", 0)
grid_gdf.insert(2, "maxx", 0)
grid_gdf.insert(3, "maxy", 0)
grid_gdf["minx"] = grid_gdf[["geometry"]].apply(lambda poly: poly.geometry.bounds[0], axis = 1)
grid_gdf["miny"] = grid_gdf[["geometry"]].apply(lambda poly: poly.geometry.bounds[1], axis = 1)
grid_gdf["maxx"] = grid_gdf[["geometry"]].apply(lambda poly: poly.geometry.bounds[2], axis = 1)
grid_gdf["maxy"] = grid_gdf[["geometry"]].apply(lambda poly: poly.geometry.bounds[3], axis = 1)
grid_gdf.sort_values(by=['maxx'])
grid_gdf['maxx'].value_counts()

maxx
77.410593    407
77.410141    407
77.409690    407
77.409239    404
77.411044    403
            ... 
81.335660      1
78.080027      1
77.452260      1
77.415699      1
80.837470      1
Name: count, Length: 13630, dtype: int64

In [12]:
buildings_df = pd.read_parquet(parquet_path, columns=['LATITUDE', 'LONGITUDE', 'AREA_IN_METERS', 'HEIGHT'])

#buildings_gdf = gpd.GeoDataFrame(buildings_df, geometry=buildings_df['POLYGON_COORDINATES'].apply(shapely.wkt.loads))
#buildings_gdf = gpd.GeoDataFrame(buildings_df, geometry=gpd.GeoSeries.from_wkb(buildings_df["POLYGON_COORDINATES"]), crs="EPSG:4326")
print(buildings_df.columns)

def compute_building_stats(cell_miny, cell_maxy, universe):
    #print(cell.bounds[0],cell.bounds[2],cell.bounds[1],cell.bounds[3])
    subset = universe[(cell_miny <= universe.LATITUDE) & (universe.LATITUDE  <= cell_maxy)]
    if subset.empty:
        return {
            "count": 0,
            "avg_area": 0,
            "max_area": 0,
            "avg_height": 0
        }
    return {
        "count": len(subset),
        "avg_area": subset["AREA_IN_METERS"].mean(skipna=True),
        "max_area": subset["AREA_IN_METERS"].max(skipna=True),
        "avg_height": subset["HEIGHT"].mean(skipna=True)
    }

results = []
curr_maxx = 0
slice_df = []
for _, row in tqdm(grid_gdf.iterrows(), total=len(grid_gdf), desc="Computing building stats"):
    if row.maxx != curr_maxx:
        slice_df = buildings_df[(row.minx <= buildings_df.LONGITUDE) & (buildings_df.LONGITUDE <= row.maxx)]
        curr_maxx = row.maxx
    stats = compute_building_stats(row.miny, row.maxy, slice_df)
    results.append(stats)

stats_gdf = gpd.GeoDataFrame(results)
grid_gdf = pd.concat([grid_gdf, stats_gdf], axis=1)

print(grid_gdf.head())

Index(['LATITUDE', 'LONGITUDE', 'AREA_IN_METERS', 'HEIGHT'], dtype='object')


Computing building stats: 100%|██████████| 1047334/1047334 [23:54<00:00, 730.13it/s] 


        minx       miny       maxx       maxy  \
0  78.204072  26.689588  78.204523  26.690039   
1  78.204072  26.690039  78.204523  26.690490   
2  78.204523  26.689588  78.204975  26.690039   
3  78.204523  26.690039  78.204975  26.690490   
4  78.204523  26.690490  78.204975  26.690941   

                                            geometry  count  avg_area  \
0  POLYGON ((78.20452 26.68959, 78.20452 26.69004...      0       0.0   
1  POLYGON ((78.20452 26.69004, 78.20452 26.69049...      0       0.0   
2  POLYGON ((78.20497 26.68959, 78.20497 26.69004...      0       0.0   
3  POLYGON ((78.20497 26.69004, 78.20497 26.69049...      0       0.0   
4  POLYGON ((78.20497 26.69049, 78.20497 26.69094...      0       0.0   

   max_area  avg_height  
0       0.0         0.0  
1       0.0         0.0  
2       0.0         0.0  
3       0.0         0.0  
4       0.0         0.0  


In [13]:
real_df = grid_gdf
real_df_adjusted = real_df.drop(
    columns=[
        "geometry",
        "minx",
        "miny",
        "maxx",
        "maxy"
    ]
)

real_df_cleaned = real_df_adjusted.fillna(0)

# print(real_df_cleaned.columns)
# print(real_df_cleaned.head(5))

# Normalize data
real_df_cleaned["avg_area_norm"] = real_df_cleaned["avg_area"] / 3000
real_df_cleaned.loc[real_df_cleaned["avg_area"] > 3000, "avg_area_norm"] = 1

real_df_cleaned["max_area_norm"] = real_df_cleaned["max_area"] / 3000
real_df_cleaned.loc[real_df_cleaned["max_area"] > 3000, "max_area_norm"] = 1

real_df_cleaned["avg_height_norm"] = real_df_cleaned["avg_height"] / 100
real_df_cleaned.loc[real_df_cleaned["avg_height"] > 100, "avg_height_norm"] = 1


real_df_cleaned_adj = real_df_cleaned.drop(columns=["avg_area", "max_area", "avg_height"])
print(real_df_cleaned_adj)

         count  avg_area_norm  max_area_norm  avg_height_norm
0            0       0.000000       0.000000            0.000
1            0       0.000000       0.000000            0.000
2            0       0.000000       0.000000            0.000
3            0       0.000000       0.000000            0.000
4            0       0.000000       0.000000            0.000
...        ...            ...            ...              ...
1047329      1       0.051281       0.051281            0.045
1047330      0       0.000000       0.000000            0.000
1047331      0       0.000000       0.000000            0.000
1047332      0       0.000000       0.000000            0.000
1047333      0       0.000000       0.000000            0.000

[1047334 rows x 4 columns]


In [14]:
print(real_df_cleaned_adj['count'].describe())
print(real_df_cleaned_adj['avg_area_norm'].describe())
print(real_df_cleaned_adj['max_area_norm'].describe())
print(real_df_cleaned_adj['avg_height_norm'].describe())

count    1.047334e+06
mean     3.320747e+00
std      5.031798e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      5.000000e+00
max      4.000000e+01
Name: count, dtype: float64
count    1.047334e+06
mean     2.367421e-02
std      5.953157e-02
min      0.000000e+00
25%      0.000000e+00
50%      8.595533e-03
75%      3.042513e-02
max      1.000000e+00
Name: avg_area_norm, dtype: float64
count    1.047334e+06
mean     4.012407e-02
std      7.942541e-02
min      0.000000e+00
25%      0.000000e+00
50%      9.779017e-03
75%      5.665726e-02
max      1.000000e+00
Name: max_area_norm, dtype: float64
count    1.047334e+06
mean     3.138101e-02
std      3.375789e-02
min      0.000000e+00
25%      0.000000e+00
50%      4.500000e-02
75%      5.500000e-02
max      7.650000e-01
Name: avg_height_norm, dtype: float64


In [15]:
# Load feature columns from training
#feature_columns = joblib.load("model/feature_columns.pkl")
#print(feature_columns)

# Align columns with training data
#new_df = real_df_cleaned_adj.reindex(
#    columns=feature_columns, fill_value=0
#)  # Ensure same structure

print(real_df_cleaned_adj.head(10))

# Load the scaler
scaler = joblib.load("model/scaler.save")
print(scaler)

# Scale using the SAME scaler (don’t fit again)
new_scaled = scaler.transform(real_df_cleaned_adj)
new_scaled

   count  avg_area_norm  max_area_norm  avg_height_norm
0      0            0.0            0.0              0.0
1      0            0.0            0.0              0.0
2      0            0.0            0.0              0.0
3      0            0.0            0.0              0.0
4      0            0.0            0.0              0.0
5      0            0.0            0.0              0.0
6      0            0.0            0.0              0.0
7      0            0.0            0.0              0.0
8      0            0.0            0.0              0.0
9      0            0.0            0.0              0.0
StandardScaler()


array([[-0.86933781, -0.59985466, -0.81350394, -0.98895246],
       [-0.86933781, -0.59985466, -0.81350394, -0.98895246],
       [-0.86933781, -0.59985466, -0.81350394, -0.98895246],
       ...,
       [-0.86933781, -0.59985466, -0.81350394, -0.98895246],
       [-0.86933781, -0.59985466, -0.81350394, -0.98895246],
       [-0.86933781, -0.59985466, -0.81350394, -0.98895246]])

In [16]:
model = keras.models.load_model(model_file_name)
pred_probs = model.predict(new_scaled)

percentages = (pred_probs * 100).round(2).flatten()  

pred_classes = (pred_probs > 0.6).astype(int)  

c:\Users\A44505693\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32730/32730 ━━━━━━━━━━━━━━━━━━━━ 58s 2ms/step


In [17]:
predicted_classes = ["formal" if label == 0 else "informal" for label in pred_classes]
real_df["prediction"] = predicted_classes
real_df["confidence"] = pred_probs

real_data_adjusted_df = real_df
real_data_adjusted_df

,minx,miny,maxx,maxy,geometry,count,avg_area,max_area,avg_height,prediction,confidence
0,78.204072,26.689588,78.204523,26.690039,"POLYGON ((78.20452 26.68959, 78.20452 26.69004...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1,78.204072,26.690039,78.204523,26.690490,"POLYGON ((78.20452 26.69004, 78.20452 26.69049...",0,0.0000,0.0000,0.0,formal,1.805678e-35
2,78.204523,26.689588,78.204975,26.690039,"POLYGON ((78.20497 26.68959, 78.20497 26.69004...",0,0.0000,0.0000,0.0,formal,1.805678e-35
3,78.204523,26.690039,78.204975,26.690490,"POLYGON ((78.20497 26.69004, 78.20497 26.69049...",0,0.0000,0.0000,0.0,formal,1.805678e-35
4,78.204523,26.690490,78.204975,26.690941,"POLYGON ((78.20497 26.69049, 78.20497 26.69094...",0,0.0000,0.0000,0.0,formal,1.805678e-35
...,...,...,...,...,...,...,...,...,...,...,...
1047329,76.250659,21.338408,76.251110,21.338859,"POLYGON ((76.25111 21.33841, 76.25111 21.33886...",1,153.8444,153.8444,4.5,formal,3.534718e-32
1047330,76.251110,21.305018,76.251561,21.305470,"POLYGON ((76.25156 21.30502, 76.25156 21.30547...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1047331,76.251110,21.337505,76.251561,21.337957,"POLYGON ((76.25156 21.33751, 76.25156 21.33796...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1047332,76.251110,21.337957,76.251561,21.338408,"POLYGON ((76.25156 21.33796, 76.25156 21.33841...",0,0.0000,0.0000,0.0,formal,1.805678e-35


In [18]:
#toto bude upravovane

#real_data_filtered_df = real_data_adjusted_df[
#    real_data_adjusted_df["prediction"] == "informal"
#]
real_data_filtered_df = real_data_adjusted_df
real_data_filtered_df

,minx,miny,maxx,maxy,geometry,count,avg_area,max_area,avg_height,prediction,confidence
0,78.204072,26.689588,78.204523,26.690039,"POLYGON ((78.20452 26.68959, 78.20452 26.69004...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1,78.204072,26.690039,78.204523,26.690490,"POLYGON ((78.20452 26.69004, 78.20452 26.69049...",0,0.0000,0.0000,0.0,formal,1.805678e-35
2,78.204523,26.689588,78.204975,26.690039,"POLYGON ((78.20497 26.68959, 78.20497 26.69004...",0,0.0000,0.0000,0.0,formal,1.805678e-35
3,78.204523,26.690039,78.204975,26.690490,"POLYGON ((78.20497 26.69004, 78.20497 26.69049...",0,0.0000,0.0000,0.0,formal,1.805678e-35
4,78.204523,26.690490,78.204975,26.690941,"POLYGON ((78.20497 26.69049, 78.20497 26.69094...",0,0.0000,0.0000,0.0,formal,1.805678e-35
...,...,...,...,...,...,...,...,...,...,...,...
1047329,76.250659,21.338408,76.251110,21.338859,"POLYGON ((76.25111 21.33841, 76.25111 21.33886...",1,153.8444,153.8444,4.5,formal,3.534718e-32
1047330,76.251110,21.305018,76.251561,21.305470,"POLYGON ((76.25156 21.30502, 76.25156 21.30547...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1047331,76.251110,21.337505,76.251561,21.337957,"POLYGON ((76.25156 21.33751, 76.25156 21.33796...",0,0.0000,0.0000,0.0,formal,1.805678e-35
1047332,76.251110,21.337957,76.251561,21.338408,"POLYGON ((76.25156 21.33796, 76.25156 21.33841...",0,0.0000,0.0000,0.0,formal,1.805678e-35


In [19]:
real_data_filtered_df["polygon"] = real_data_filtered_df["geometry"]
real_data_filtered_df.to_parquet(f"{name}_neuralNetwork_informal_detection.parquet")

In [ ]:
# Build GeoJSON features


color_map = {
    "high": "#00cd00", # green
    "mid": "#ff4d4d", # red
    "low": "#1591EA", # blue
}     

features = []
for _, row in real_data_filtered_df.iterrows():
    print(row)

    confidence = row["confidence"]
    color = (
        color_map["high"] if confidence > 0.6
        else color_map["low"]
    )

    
    feature = {
        "type": "Feature",
        "properties": {
            "confidence": row["confidence"],
            "class": row["prediction"],
            # "trees": row["num_pixels_ge_3m"],
            "fill": color,
            "stroke": color
        },
        "geometry": mapping(row["polygon"]),
    }
    features.append(feature)

In [34]:
# Save to file
import json
# Wrap in a FeatureCollection
geojson_dict = {"type": "FeatureCollection", "features": features}

output_file = f"{name}_neuralNetwork_50_infromal_detection.geojson"
with open(output_file,
    "w",
) as f:
    json.dump(geojson_dict, f, indent=2)